In [4]:
import cv2
import numpy as np

def find_bounding_boxes(frame, kernel_size=3):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_blurred = cv2.GaussianBlur(frame_gray, (kernel_size, kernel_size), 5)

    _, binary_frame = cv2.threshold(frame_blurred, 1, 255, cv2.THRESH_BINARY)
    _, _, stats, _ = cv2.connectedComponentsWithStats(binary_frame, connectivity=8)

    bounding_boxes = []
    for stat in stats[1:]:
        x, y, w, h = stat[:4]
        if w > 5 and h > 5:
            bounding_boxes.append((x, y, w, h))

    filtered_boxes = [box for box in bounding_boxes if box[2] >= 10 and box[3] >= 10]
    return filtered_boxes


def draw_word_box(frame, bounding_boxes):
    for box in bounding_boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
    return frame

def sort_by_y(box):
    return box[1]

def group_words_into_line(bounding_boxes, max_vertical_gap=10):
    if not bounding_boxes:
        return []

    grouped_lines = []
    current_line = []
    
    bounding_boxes.sort(key=sort_by_y)

    for box in bounding_boxes:
        if not current_line or box[1] - current_line[-1][1] <= max_vertical_gap:
            current_line.append(box)
        else:
            grouped_lines.append(current_line)
            current_line = [box]

    grouped_lines.append(current_line)
    return grouped_lines

def draw_line_box(frame, grouped_lines, padding=5):
    for line in grouped_lines:
        x_min = min(box[0] for box in line)
        y_min = min(box[1] for box in line)
        x_max = max(box[0] + box[2] for box in line)
        y_max = max(box[1] + box[3] for box in line)
        cv2.rectangle(frame, (x_min - padding, y_min - padding), (x_max + padding, y_max + padding), (0, 0, 255), 1)
    return frame

# Read the video
video_path = r"F:\Games\Project Video.mp4"
cap = cv2.VideoCapture(video_path)



while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    width_percentage = 0.78  # Example width percentage
    height_percentage = 0.15  # Example height percentage

    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([179, 15, 255], dtype=np.uint8)
    height, width, _ = frame.shape
    # Calculate the ROI dimensions and coordinates
    roi_width = int(width * width_percentage)
    roi_height = int(height * height_percentage)
    roi_x = int((width - roi_width) / 2)
    roi_y = int(height - roi_height)
    roi = frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_white, upper_white)
    black_frame = np.zeros_like(frame)
    black_frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width][mask != 0] = roi[mask != 0]
    line = black_frame
    kernel = np.ones((1,15), np.uint8)
    black_frame_line = cv2.dilate(black_frame, kernel, iterations=1)
    word = black_frame_line
    filtered_word = find_bounding_boxes(word)
    filtered_line = find_bounding_boxes(line)
    lines = group_words_into_line(filtered_line)
    line_red_frame = draw_line_box(frame.copy(), lines)
    final_frame = draw_word_box(line_red_frame, filtered_word)

    # Convert the frame to HSV color space

    # Get the dimensions of the frame


    # Display the result
    cv2.imshow('Result', final_frame)


    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()
